In [1]:
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from collections import Counter
from pymystem3 import Mystem
import re
import numpy as np

In [2]:
habr_clusters = pd.read_csv('/Users/alexandrasorokovikova/Data Science/НИР Сравнение IT/Comparison-of-IT-trends/clustering/habs_clusters.csv', index_col=0)
habr_info = pd.read_json('/Users/alexandrasorokovikova/Data Science/НИР Сравнение IT/Comparison-of-IT-trends/data_parsing/data/habr.json')


In [3]:
habr_info

,time,tags,habs,saved,comments,views
0,2016-04-11T09:29:03.000Z,"[безопасная динамичность, brave, блокировка ре...",[управление медиа],4,0,1749
1,2016-04-09T14:19:17.000Z,"[dash, блокчейн, криптовалюта, управление]","[фриланс, венчурные инвестиции]",12,7,10370
2,2016-04-10T05:11:42.000Z,"[mail.ru group, headhunter, российский рынок, ...","[карьера в it-индустрии, бизнес-модели]",12,1,5472
3,2016-04-11T17:05:54.000Z,"[tidal, jay-z, канье уэст, стриминговые сервисы]",[монетизация веб-сервисов],3,1,3706
4,2016-04-09T14:52:55.000Z,"[mozilla, brave, блокировщики рекламы, судебно...",[законодательство в it],4,7,6244
...,...,...,...,...,...,...
38066,2021-11-11T07:34:51.000Z,[cisco],"[блог компании cloud4y, информационная безопас...",3,1,2098
38067,2021-11-11T08:22:05.000Z,"[георезервирование, резервирование каналов, dr...","[it-инфраструктура, сетевые технологии, облачн...",6,2,773
38068,2021-11-11T11:13:02.000Z,"[газоразрядный индикатор, имг-1-02, гип-10000,...","[программирование микроконтроллеров, старое же...",61,37,11593
38069,2021-11-11T11:55:58.000Z,"[минцифры, госуслуги, ddos, атака]","[it-инфраструктура, законодательство в it, it-...",1,13,3796


In [4]:
habr_clusters.head()

,hab,agglomerative_cluster
0,компания,1
1,блог,1
2,разработка,7
3,программирование,4
4,веб,7


## Выделение категорий
Для выделения категории из кластеров выберем наиболее встречающийся hab

In [5]:
stem = Mystem()

def count_categories(items, data):
    habs_counter = Counter()
    habs_cleaned = []
    
    for row in tqdm(data.index):
        
        hab_data = []
        
        habs_data = data[row]
        habs_data = ' '.join(stem.lemmatize(' '.join(habs_data)))
        habs_data = re.sub("[^A-Za-zА-Яа-я\s]", '', habs_data)
        habs_data = habs_data.split()
        
        for item in items:
            if len(item.split('_')) == 2:
                item_splited = item.split('_')
                if item_splited[0] in habs_data and item_splited[1] in habs_data:
                    habs_counter[item] += 1
                    hab_data.append(item)
            else:
                if item in habs_data:
                    habs_counter[item] += 1
                    hab_data.append(item)
                    
        habs_cleaned.append(hab_data)
                    
    return habs_counter, habs_cleaned
        
habs_counter, habs_cleaned = count_categories(habr_clusters['hab'], habr_info['habs'])

100%|██████████| 38071/38071 [00:34<00:00, 1105.85it/s]


In [6]:
habs_counter

Counter({'управление': 3223,
         'медиа': 145,
         'венчурный_инвестиция': 157,
         'фриланс': 146,
         'it': 9074,
         'карьера': 1621,
         'индустрия': 1617,
         'бизнес_модель': 469,
         'бизнес': 470,
         'веб': 3028,
         'сервис': 1292,
         'монетизация': 300,
         'законодательство': 1527,
         'финансы': 945,
         'патентование': 64,
         'интернет': 860,
         'маркетинг': 558,
         'контекстный_реклама': 108,
         'gtd': 211,
         'исследование_прогноз': 750,
         'исследование': 750,
         'повышение_конверсия': 143,
         'контент': 196,
         'growth_hacking': 80,
         'проект': 1181,
         'продукт': 814,
         'конференция': 887,
         'система': 2712,
         'игра': 1935,
         'e_commerce': 420,
         'e': 420,
         'читальный_зал': 904,
         'продажа': 232,
         'аналитика': 159,
         'учебный_процесс': 888,
         'процесс': 888,
  

In [7]:
habr_info['habs'] = habs_cleaned

In [8]:
def count_habs(row):
    return habs_counter[row]

habr_clusters['frequency'] = habr_clusters['hab'].apply(count_habs)
habr_clusters

,hab,agglomerative_cluster,frequency
0,компания,1,2328
1,блог,1,497
2,разработка,7,8178
3,программирование,4,5184
4,веб,7,3028
...,...,...,...
1704,1c,1,0
1705,smmframe,0,0
1706,связной,0,0
1707,экспресс,0,0


In [9]:
habr_clusters[habr_clusters['agglomerative_cluster'] == 4].sort_values(by = 'frequency', ascending = False).head(10)

,hab,agglomerative_cluster,frequency
3,программирование,4,5184
8,система,4,2712
9,c,4,1668
37,python,4,1451
25,open,4,1335
28,source,4,1332
26,проектирование,4,1174
19,алгоритм,4,1152
32,java,4,1035
30,тестирование,4,977


### Для каждого кластера выберем название
**2**: Железо, электроника

**3**: Сетевые технологии, администрирование, devops

**4**: Проектирование, тестирование

**5**: Информационная безопасность

**6**: Маркетинг, контекстная реклама

**7**: Верстка, html, javascript

**8**: Машинное обучение

**9**: E-commerce, финансы

**10**: Дизайн интерфейсов

**11**: Бизнес, CRM

**12**: SEO

Для категории **0** и **1** не получилось выделить центральный объект (встречаемость хабов в этих категориях очень маленькая, так что ее лучше не учитывать).

## Разметим статьи их категориями

In [10]:
fields = {2 : 'Железо, электроника',
         3 : 'Сетевые технологии, администрирование, devops',
         4 : 'Проектирование, тестирование',
         5 : 'Информационная безопасность',
         6 : 'Маркетинг, контекстная реклама',
         7 : 'Верстка, html, javascript',
         8 : 'Машинное обучение',
         9 : 'E-commerce, финансы',
         10 : 'Дизайн интерфейсов',
         11 : 'Бизнес, CRM',
         12 : 'SEO'}

In [15]:
def get_field(row):
    field = []
    for hab in row:
        field.append(habr_clusters[habr_clusters['hab'] == hab]['agglomerative_cluster'].values[0])
    return field

habr_info['field'] = habr_info['habs'].apply(get_field)
habr_info

,time,tags,habs,saved,comments,views,field
0,2016-04-11T09:29:03.000Z,"[безопасная динамичность, brave, блокировка ре...","[управление, медиа]",4,0,1749,"[11, 6]"
1,2016-04-09T14:19:17.000Z,"[dash, блокчейн, криптовалюта, управление]","[венчурный_инвестиция, фриланс]",12,7,10370,"[11, 11]"
2,2016-04-10T05:11:42.000Z,"[mail.ru group, headhunter, российский рынок, ...","[it, карьера, индустрия, бизнес_модель, бизнес]",12,1,5472,"[9, 11, 11, 11, 11]"
3,2016-04-11T17:05:54.000Z,"[tidal, jay-z, канье уэст, стриминговые сервисы]","[веб, сервис, монетизация]",3,1,3706,"[7, 12, 6]"
4,2016-04-09T14:52:55.000Z,"[mozilla, brave, блокировщики рекламы, судебно...","[it, законодательство]",4,7,6244,"[9, 11]"
...,...,...,...,...,...,...,...
38066,2021-11-11T07:34:51.000Z,[cisco],"[компания, блог, информационный, безопасность,...",3,1,2098,"[1, 1, 5, 5, 3, 2]"
38067,2021-11-11T08:22:05.000Z,"[георезервирование, резервирование каналов, dr...","[it, технология, инфраструктура, сетевой, серв...",6,2,773,"[9, 3, 3, 3, 12, 2]"
38068,2021-11-11T11:13:02.000Z,"[газоразрядный индикатор, имг-1-02, гип-10000,...","[программирование, микроконтроллер, старый_жел...",61,37,11593,"[4, 4, 2, 2, 2]"
38069,2021-11-11T11:55:58.000Z,"[минцифры, госуслуги, ddos, атака]","[компания, it, инфраструктура, законодательство]",1,13,3796,"[1, 9, 3, 11]"


In [23]:
habr_info.to_json('habr_info.json')